In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Flatten, Input, Concatenate, concatenate,Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import LabelEncoder
from tqdm.keras import TqdmCallback
import tensorflow.keras.utils as utils
from datetime import datetime

import cv2
import zipfile
import shutil
import os
import glob, json

from tensorflow.keras.applications import Xception, InceptionV3, MobileNet, ResNet50

import random
import os
from tqdm import tqdm

from ultralytics import YOLO

In [4]:
from ultralytics import YOLO
## yolo 모델 설정
model = YOLO('yolov8n.pt')

results = model.train(data='D:\\smartfarm_tomato\\data_for_yolo\\data.yaml', 
                      epochs=1,
                     imgsz=640,
                     batch=-1)

results = model.val()

New https://pypi.org/project/ultralytics/8.0.123 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.119  Python-3.9.13 torch-2.0.1+cpu CPU
yolo\engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=D:\smartfarm_tomato\data_for_yolo\data.yaml, epochs=1, patience=50, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_mask

In [24]:
# Dictionary mapping type to class id
type_to_id = { 'd1': 2, 'd2': 2, 'f': 3, 'e': 4}

# Input directories and output directory
input_dirs = ['d1.꽃(개화군)', 'd2.꽃(착과군)', 'e.열매', 'f.만개꽃']
output_dir = 'D:\\smartfarm_tomato\\data_for_yolo\\2.Validation\\label'
base_dir = 'D:\\smartfarm_tomato\\data_for_yolo\\2.Validation\\label'

# Function to convert polygon to bounding box
def poly_to_bbox(poly_points):
    xs = [point[0] for point in poly_points]
    ys = [point[1] for point in poly_points]
    
    x1 = min(xs)
    y1 = min(ys)
    x2 = max(xs)
    y2 = max(ys)
    
    return [[x1, y1], [x2, y2]]

# Loop over input directories
for filename in tqdm(os.listdir(base_dir), desc="Processing directories"):
    if filename.endswith('.json'):
        # Open JSON file
        with open(os.path.join(base_dir,filename)) as f:
            data = json.load(f)

            # Create directory if not exists
#             os.makedirs(os.path.join(output_dir, input_dir), exist_ok=True)

            # Create output .txt file
            with open(os.path.join(output_dir, os.path.splitext(filename)[0] + '.txt'), 'w') as outfile:
                # Loop over shapes in data
                for shape in data['shapes']:
                    # Get class id from type
                    class_id = type_to_id[data['file_attributes']["type"]]

                    # Convert polygon to bounding box
                    [x1, y1], [x2, y2] = poly_to_bbox(shape['points'])

                    # Calculate YOLO format values
                    x_center = (x1 + x2) / 2.0 / data['imageWidth']
                    y_center = (y1 + y2) / 2.0 / data['imageHeight']
                    width = abs(x2 - x1) / data['imageWidth']
                    height = abs(y2 - y1) / data['imageHeight']

                    # Write to output file
                    outfile.write(f'{class_id} {x_center} {y_center} {width} {height}\n')

                    
# 변환 끝난 것 삭제 
for filename in tqdm(os.listdir(base_dir), desc="Processing directories"):
    if filename.endswith('.json'):
        os.remove(os.path.join(base_dir,filename))

Processing directories: 100%|█████████████████████████████████████████████████| 21640/21640 [00:00<00:00, 24616.80it/s]


In [2]:
import os
import re

log = r"""
train: WARNING  D:\smartfarm_tomato\data_for_yolo\images\train\images\V001_tom3_45_074_e_12_20211113_17_06111810_24085170.png: 1 duplicate labels removed
train: WARNING  D:\smartfarm_tomato\data_for_yolo\images\train\images\V001_tom3_48_062_e_09_20211202_20_06103436_24085170.png: 1 duplicate labels removed
train: WARNING  D:\smartfarm_tomato\data_for_yolo\images\train\images\V001_tom4_41_026_d1_06_20211016_10_05104940_66695123.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0369]
train: WARNING  D:\smartfarm_tomato\data_for_yolo\images\train\images\V001_tom4_41_027_d1_06_20211016_10_05110638_66695123.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0087]
train: WARNING  D:\smartfarm_tomato\data_for_yolo\images\train\images\V001_tom4_41_030_d2_06_20211016_10_05121251_66695123.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0571      1.2942      1.1818]
train: WARNING  D:\smartfarm_tomato\data_for_yolo\images\train\images\V001_tom4_41_031_d2_07_20211016_10_05122715_66695123.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1189      1.0409]
train: WARNING  D:\smartfarm_tomato\data_for_yolo\images\train\images\V001_tom4_41_041_d1_07_20211016_10_05150225_66695123.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1155]
train: WARNING  D:\smartfarm_tomato\data_for_yolo\images\train\images\V001_tom4_41_041_d1_07_20211016_10_06150225_66695123.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0817]
train: WARNING  D:\smartfarm_tomato\data_for_yolo\images\train\images\V001_tom6_44_024_d2_05_20211103_09_05145351_37664584.png: 1 duplicate labels removed
val: WARNING  D:\smartfarm_tomato\data_for_yolo\images\valid\images\V001_tom3_49_093_d2_15_20211212_21_04103900_47485660.png: 1 duplicate labels removed
val: WARNING  D:\smartfarm_tomato\data_for_yolo\images\valid\images\V001_tom3_50_070_e_11_20211217_22_05093720_24085170.png: 3 duplicate labels removed
val: WARNING  D:\smartfarm_tomato\data_for_yolo\images\valid\images\V001_tom4_41_041_d1_07_20211016_10_07150225_66695123.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.2503      1.0795      1.1435]
"""

# 이미지 파일 경로 추출하기
image_paths = re.findall(r'D:[^:]+\.png', log)

# 각 이미지 파일에 대해서
for image_path in image_paths:
    # 파일 경로에서 \를 /로 변환
    image_path = image_path.replace("\\", "/")
    
    # 비출력 가능한 문자를 제거
    image_path = ''.join(ch for ch in image_path if ch.isprintable())
    
    # 이미지 파일이 실제로 존재하는 경우 삭제
    if os.path.exists(image_path):
        os.remove(image_path)
        print(f"Deleted image file: {image_path}")
    else:
        print(f"No such image file: {image_path}")
    
    # 레이블 파일 경로 생성
    label_path = image_path.replace("/images/", "/labels/").replace(".png", ".txt")
    
    # 레이블 파일이 실제로 존재하는 경우 삭제
    if os.path.exists(label_path):
        os.remove(label_path)
        print(f"Deleted label file: {label_path}")
    else:
        print(f"No such label file: {label_path}")

Deleted image file: D:/smartfarm_tomato/data_for_yolo/images/train/images/V001_tom3_45_074_e_12_20211113_17_06111810_24085170.png
No such label file: D:/smartfarm_tomato/data_for_yolo/labels/train/labels/V001_tom3_45_074_e_12_20211113_17_06111810_24085170.txt
Deleted image file: D:/smartfarm_tomato/data_for_yolo/images/train/images/V001_tom3_48_062_e_09_20211202_20_06103436_24085170.png
No such label file: D:/smartfarm_tomato/data_for_yolo/labels/train/labels/V001_tom3_48_062_e_09_20211202_20_06103436_24085170.txt
Deleted image file: D:/smartfarm_tomato/data_for_yolo/images/train/images/V001_tom4_41_026_d1_06_20211016_10_05104940_66695123.png
No such label file: D:/smartfarm_tomato/data_for_yolo/labels/train/labels/V001_tom4_41_026_d1_06_20211016_10_05104940_66695123.txt
Deleted image file: D:/smartfarm_tomato/data_for_yolo/images/train/images/V001_tom4_41_027_d1_06_20211016_10_05110638_66695123.png
No such label file: D:/smartfarm_tomato/data_for_yolo/labels/train/labels/V001_tom4_41_

In [0]:
### normal sick yolo test
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

results = model.train(data='D:\\smartfarm_tomato\\data_for_yolo\\data\\inspection\\data.yaml', 
                      epochs=1,
                     imgsz=32,
                     batch=-1)

results = model.val()

New https://pypi.org/project/ultralytics/8.0.123 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.119  Python-3.9.13 torch-2.0.1+cpu CPU
yolo\engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=D:\smartfarm_tomato\data_for_yolo\data\inspection\data.yaml, epochs=1, patience=50, batch=-1, imgsz=32, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=No

KeyboardInterrupt: 